# CCN 2022 Tutorial
This notebook was designed to accompany the [Varieties of human-like AI tutorial](https://2022.ccneuro.org/view_event.php?mid=18) which took place during CCN 2022. 

It also serves as an interactive introduction to various topics in reinforcement learning, and may be of general interest.

In [ ]:
# Uncomment below lines and run this cell to install neuronav and requirements for colab.

#!git clone https://github.com/awjuliani/neuro-nav
#!pip install ./neuro-nav[experiments_remote]

In [ ]:
from neuronav.envs.grid_env import GridEnv, GridSize, GridObsType, OrientationType
from neuronav.envs.graph_env import GraphEnv
from neuronav.agents.td_agents import TDSR, TDQ
from neuronav.agents.mb_agents import MBV
from neuronav.envs.grid_topographies import GridTopography
from neuronav.envs.graph_structures import GraphStructure
from neuronav.utils import run_episode
from neuronav.notebook_helpers import plot_values_and_policy
import matplotlib.pyplot as plt
import numpy as np

## The reinforcement learning loop

In [ ]:
env = GraphEnv(graph_structure=GraphStructure.two_step)
reward_locs = {3: 0.5, 4: -0.5, 5: 1.0, 6: -1.0}
obs = env.reset(reward_locs=reward_locs)
env.render()

## Q-Learning in bandit tasks

## Q-learning in multi-step tasks

## The successor representation

## Experience replay with Dyna

## Model-based algorithms

## Next steps